读取数据

In [1]:
from tools.data_utils import read_jsonl_file

train_data = read_jsonl_file('./data/cluener/conll_format/train.jsonl')
dev_data = read_jsonl_file('./data/cluener/conll_format/dev.jsonl')

/home/hejiansu/anaconda3/envs/deepner/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tools.data_utils import get_unique_tags

unique_tags, labels_to_ids, ids_to_labels = get_unique_tags(train_data)

In [3]:
import numpy as np # linear algebra
# data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [4]:
from tools.data import SequenceLabelingDataset

train_dataset = SequenceLabelingDataset(data=train_data, labels_to_ids=labels_to_ids, tokenizer=tokenizer, max_length=128)

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [6]:
from transformers import BertForTokenClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [68]:
from tqdm import tqdm


for batch in tqdm(train_dataloader):
    input_ids = batch['input_ids']
    attention_masks = batch['attention_mask']
    labels = batch['labels']

    pred = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
    break

  0%|          | 0/336 [00:02<?, ?it/s]


In [73]:
active_loss = attention_masks.view(-1) == 1

In [74]:
num_classes = pred.logits.shape[-1]
num_classes

21

In [75]:
active_logits = pred.logits.view(-1, num_classes)[active_loss]
active_logits.shape

torch.Size([1225, 21])

In [76]:
active_labels = labels.view(-1)[active_loss]
active_labels.shape

torch.Size([1225])

In [80]:
torch.argmax(active_logits, dim=-1).shape

torch.Size([1225])

In [63]:
labels = labels.contiguous().view(-1)
pred = torch.argmax(logits, dim=-1).contiguous().view(-1)

In [64]:
from sklearn.metrics import precision_recall_fscore_support, f1_score

label_result = [labels]
pred_result = [pred]
label_result = torch.cat(label_result)
pred_result = torch.cat(pred_result)
print(label_result.shape)
print(pred_result.shape)

torch.Size([4096])
torch.Size([4096])


In [65]:
p, r, f1, _ = precision_recall_fscore_support(label_result.cpu().numpy(),
                                                    pred_result.cpu().numpy(),
                                                    average="macro")

/home/hejiansu/anaconda3/envs/deepner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
f1

0.008815544635429233